<img src="imgs/unicamp.png" width="150" height="150">

## MO444/MC886 - Aprendizado de Máquina e Reconhecimento de Padrões

Esse trabalho foi feito pelos seguintes membros:

- Germán Darío Buitrago Salazar - 164321
- Giovanna Relvas Bartilotti - 023907
- Lucas Zanco Ladeira - 188951
- Rafael Scherer - 204990 

O código original deste projeto está disponível no [repositório do Github](https://github.com/lucaslzl/mo444_p4_qea).

Vídeo de apresentação [link](https://youtube.com).

## Trabalho Final - Serviço de FAQ para os Objetivos de Desenvolvimento Sustentável

<b>Declaração do Problema</b>

"Este trabalho tem como objetivo construir um sistema de Aprendizado de Máquina para solucionar um problema escolhido pelo grupo inserido em um dos Objetivos de Desenvolvimento Sustentável (ODS) da ONU. O trabalho do seu grupo é encontrar uma solução adequada para o problema escolhido."

Neste trabalho foi construído um serviço de FAQ para tirar dúvidas das pessoas sobre os Objetivos de Desenvolvimento Sustentável. Para tal, foram utilizadas algumas estratégias de processamento de linguagem natural, modelos supervisionados, e bibliotecas para auxiliar na construção da base de dados.

## 1. Cenário

De acordo com [1], desenvolvimento sustentável se refere a um modelo de desenvolvimento econômico, social e político que esteja em harmonia com o meio ambiente. Isso significa utilizar conscientemente os recursos disponíveis para que não acabe com o meio ambiente. Ele é imprescindível para o futuro da população humana e dos outros animais que habitam o planeta terra. As Nações Unidas fizeram um acordo em 2015 com 195 países para trabalhar em 17 objetivos de forma a mudar o mundo para melhor. Esses objetivos estão distribuídos em ações para acabar com a pobreza, proteger o meio ambiente e o clima e garantir que as pessoas em todos os lugares possam desfrutar de paz e prosperidade. É possível observar na figura a seguir esses objetivos.

<img src="imgs/ods_pt.png" width="700" height="700">

## 2. Base de Dados

A base de dados utilizada neste trabalho compreende 10 perguntas e respostas para cada um dos 16 primeiros objetivos. Ela foi construída manualmente pelos 4 integrantes do grupo, sendo que, cada integrante focou em 4 objetivos. As perguntas e respostas foram retiradas do relatório das Nações Unidas de 2020 [2]. Portanto, a base de dados tem 160 pares de perguntas e respostas. Para desenvolver um modelo supervisionado vimos que não seria o suficiente, pois cada resposta compreende uma classe distinta. Para resolver esse problema, uma biblioteca chamada NLPAug [3] foi utilizada, a qual tem como objetivo criar mais dados textuais a partir de uma base existente. Com ela foram criados mais 4 registros para cada par fazendo pequenas substituições nas frases, crescendo a base de dados para 800 registros. Essas substituições se referem a utilizar o modelo <i>distilbert</i> para buscar palavras próximas no embedding, e o modelo <i>wordnet</i> para buscar sinônimos de palavras das perguntas.

Para observar quais foram as perguntas criadas execute o código a seguir modificando o nome do arquivo.

In [1]:
import pandas as pd

fname = '1_poverty_eradication'
df = pd.read_excel(f'data/{fname}.xlsx')
df.dropna(inplace=True)

In [2]:
df

,Question,Answer
0,What is the objective of Sustainable Developme...,Eradicate poverty in all forms and everywhere
1,How many dollars does a person of extreme pove...,1.9
2,"What proportion of men, women and children of ...",half
3,\nHow to ensure a significant mobilization of ...,From a variety of sources
4,Why create solid policy frameworks?,to support investments
5,"By 2030, how many men will have equal rights t...",all
6,What is expected about the exposure and vulner...,reduce
7,cite an objective on implementing programs and...,End poverty in all its dimensions
8,\nWhat is the level of implementation of socia...,national
9,"For how many men and women, by 2030 will have ...",all


## 3. Solução

Todos os anos as Nações Unidas publicam um relatório [2] com o avanço dessas ações do ODS. No entanto, esta forma de publicação não é acessível e nem chamativo para a maior parte das pessoas. Sendo assim, é necessário criar uma maneira interativa de levar essas informações para a população. Um serviço de FAQ pode ajudar nesse sentido, com a característica de fazer perguntas abertamente e receber alguma resposta. Para tal, desenvolvemos esse serviço o qual utiliza processamento de linguagem natural (PLN) para identificar sobre qual objetivo o usuário deseja conhecer e qual é a resposta mais provável. 

O serviço utiliza 2 níveis modelos, o primeiro para predizer qual é o objetivo que a pergunta se refere, e o segundo qual é a provável resposta sobre o objetivo predito. Neste trabalho consideramos 16 objetivos, sendo assim, são necessários 17 modelos. A vantagem de utilizar uma estrutura de classificação hierárquica para classificar uma subclasse é que esses classificadores de subclasses são treinados especificamente para cada classe, fazendo com que ao invés de ter que classificar uma subclasse tendo 800 outras opções, cada classificador possui apenas as subclasses referentes a sua classe principal.

Antes de treinar os modelos os dados passaram por um pre-processamento para melhorar a qualidade dos dados e transformá-los em uma representação que modelos consigam entender. Primeiramente os dados foram limpos removendo caracteres especiais, como por exemplo "<i>\n</i>". Considerando que os dados são textuais, utilizamos a classe Doc2Vec da biblioteca gensim para treinar um modelo de sentence embedding e transformar os textos em vetores de 50 dimensões. A quantidade de dimensões foi escolhida com testes empíricos. 

Os dados foram duplicados e formatados de acordo com o modelo destino. Nos dados destinados a predição de qual objetivo a pergunta se refere, foi criada uma <i>feature</i> da classe que varia de 1-16 de acordo com os objetivos. No outro conjunto de dados foi criada uma <i>feature</i> da classe com cada pergunta possível, variando de 1-160. Mesmo que para cada modelo deste nível apenas sejam utilizadas 10 variações de perguntas, essa organização facilita identificar qual pergunta é em uma lista. Após, foram aplicados experimentos com diferentes modelos, comparando a configuração de hiperparâmetros padrão da biblioteca scikit-learn, e também, o tuning deles como será apresentado na seção de desenvolvimento.

## 4. Desenvolvimento

Apresentaremos aqui alguns dos algoritmos utilizados no desenvolvimento da solução, como a estratégia de tuning dos modelos. Por fim, o modelo que obteve melhor resultado nos testes. O código fonte está distribuído em diferentes notebooks sendo:
- augment: Estratégia de data augmentation
- modelling: Processamento dos dados, treinamento de modelos e criação dos gráficos com resultados
- tunning: Estratégia de tuning dos hiperparâmetros
- talk_to_me: Notebook para testar a solução

### 4.1 Data Augmentation

In [ ]:
def augment(text, times=2):

    augmented = []
        
    # Iterate x times
    for i in range(times):
        
        # Substitute with distilbert
        aug = naw.ContextualWordEmbsAug(
            model_path='distilbert-base-uncased', action="substitute")
        augmented.append(aug.augment(text))

        # Substitute with wordnet
        aug = naw.SynonymAug(aug_src='wordnet')
        augmented.append(aug.augment(text))
    
    return augmented

### 4.2 Sentence embedding

In [ ]:
# Tokenize
tokenized = [word_tokenize(c.lower()) for c in corpus]

# Tag
tagged = [TaggedDocument(d, [i]) for i, d in enumerate(tokenized)]

# Train model
model = Doc2Vec(tagged, vector_size=50, window=2, min_count=1, epochs=100)

### 4.3 Separação dos dados

In [ ]:
# Which subject?
class_sub = {}
X_sub, y_sub = [], []

for i, f in enumerate(files):

    class_sub[str(i+1)] = f
    X_sub.extend(files[f]['question'].values)
    y_sub.extend([str(i+1)]*len(files[f]['question'].values))

In [ ]:
# Which question?
class_que = {}
X_que, y_que = [], []
count = 0
last = ''

for i, f in enumerate(files):
    
    for j, row in files[f].iterrows():
        
        que = row['question']
        ans = row['answer']
        
        if last != ans:
            last = ans
            count += 1
        
        class_que[str(count)] = ans
        X_que.append(que)
        y_que.append(str(count))

### 4.4 Tuning

A busca de hiperparâmetros foi realizada aplicando o algoritmo de PSO, que otimiza iterativamente uma solução candidata aplicando uma medida de qualidade G(x). Devido a que a avaliação dos modelos classificatórios usa o f1-score como métrica de acurácia do sistema escolhida, decidiu-se otimizar dita medida minimizando seu valor. Assim, para sua aplicação, a medida foi ajeitada para achar o menor valor possível, como visto na equacionamento abaixo.

$$ G(x) = 1 - {f_1Score}$$

O resultado da busca está na tabela seguinte. Para o modelo de Random Forest, a busca foi feita para encontrar o melhor número de estimadores `n_estimators` e o melhor fator pra fazer prunning nas árvores `ccp_alpha`; no MLP foi realizada para encontrar o melhor número de neurônios `hidden_layer_sizes`; no Gradient Boosting foi usada no `learning_rate` e no número de estimadores `n_estimators`. De todos os modelos treinados, o melhor resultado foi o Random Forest ao usar 227 estimadores e `ccp_alpha` zerado. Comparado com o métrica do cross validation, o modelo melhorou em 3.2%.

| Modelo                             | Melhores Hiperparâmetros                                                         | F1-Score Cross-Validation | F1-Score Test |
|------------------------------------|----------------------------------------------------------------------------------|---------------------------|---------------|
| RandomForestClassifier_Default     | {"random_state" : 42},                                                           | 0.748775                  | 0.853009      |
| RandomForestClassifier             | {"n_estimators" : 277, "ccp_alpha" : 0.0, "random_state" : 42}                   | 0.780520                  | 0.871865      |
| MLPClassifier_Default              | {"max_iter" : 300, "random_state" : 42}                                          | 0.753435                  | 0.733811      |
| MLPClassifier                      | {"max_iter" : 200, "random_state" : 42, "hidden_layer_sizes" : 39}               | 0.726760                  | 0.704976      |
| GradientBoostingClassifier_Default | {"random_state" : 42}                                                            | 0.600771                  | 0.656114      |
| GradientBoostingClassifier         | {"learning_rate" : 0.2997228955422446, "n_estimators" : 99, "random_state" : 42} | 0.657622                  | 0.660192      |
| LogisticRegression_Default         | {"random_state" : 42}                                                            | 0.647347                  | 0.651366      |

Os resultados do tuning para as question estão na tabela a seguir. Observe que os melhores são os obtidos com o MLP, RandomForest e Logistic Regression.


| Modelo                             | Melhores Hiperparâmetros                                                         | F1-Score Cross-Validation | F1-Score Test |
|------------------------------------|----------------------------------------------------------------------------------|---------------------------|---------------|
| RandomForestClassifier_Default     | {"random_state" : 42},                                                           | 0.970370                  | 1.0           |
| RandomForestClassifier             | {"n_estimators" : 115, "ccp_alpha" : 0.0, "random_state" : 42}                   | 0.970370                  | 1.0           |
| MLPClassifier_Default              | {"max_iter" : 300, "random_state" : 42}                                          | 1.0                       | 1.0           |
| MLPClassifier                      | {"max_iter" : 200, "random_state" : 42, "hidden_layer_sizes" : 62}               | 1.0                       | 1.0           |
| GradientBoostingClassifier_Default | {"random_state" : 42}                                                            | 0.804444                  | 1.0           |
| GradientBoostingClassifier         | {"learning_rate" : 0.13227850217356052, "n_estimators" : 5, "random_state" : 42} | 0.722222                  | 1.0           |
| LogisticRegression_Default         | {"random_state" : 42}                                                            | 1.0                       | 1.0           |

## 5. Metodologia e Resultados

Um modelo de classificação tem como objetivo decidir em qual classe uma nova observação pertence dentre das classes possíveis.  A avaliação de um modelo de classificação é feita a partir da comparação entre as classes preditas pelo modelo e as classes verdadeiras de cada exemplo. Todas as métricas de classificação têm como objetivo comum medir quão distante o modelo está da classificação perfeita, porém fazem isto de formas diferentes. Cada algoritmo foi avaliado com base na F1-Score: É uma medida que considera tanto a precision quanto o recall para gerar uma pontuação, sendo a melhor medida para se avaliar um classificador.

Para selecionar o melhor modelo consideramos as seguintes opções:
- RandomForest
- Multilayer Perceptron
- Gradient Boosting
- Logistic Regression

Esses foram testados com e sem tuning dos hiperparâmetros como apresentados anteriormente, como também comparados de acordo com cada objetivo. Os resultados serão apresentados a seguir.

<table><tr><td><img src='imgs/res_sub_1.png'></td><td><img src='imgs/res_sub_2.png'></td><td><img src='imgs/res_sub_3.png'></td></tr></table>

<table><tr><td><img src='imgs/res_sub_4.png'></td><td><img src='imgs/res_sub_5.png'></td><td><img src='imgs/res_sub_6.png'></td></tr></table>

<table><tr><td><img src='imgs/res_sub_7.png'></td><td><img src='imgs/res_sub_8.png'></td><td><img src='imgs/res_sub_9.png'></td></tr></table>

<table><tr><td><img src='imgs/res_sub_10.png'></td><td><img src='imgs/res_sub_11.png'></td><td><img src='imgs/res_sub_12.png'></td></tr></table>

<table><tr><td><img src='imgs/res_sub_13.png'></td><td><img src='imgs/res_sub_14.png'></td><td><img src='imgs/res_sub_15.png'></td></tr></table>

<img src="imgs/res_sub_16.png" width="320" height="320">

É possível observar que para a maior parte dos objetivos o modelo Random Forest se saiu igual ou melhor que os outros. Com essa comparação ele foi escolhido para ser utilizado no serviço.

## Conclusões

O presente trabalho estudou a eficiência de um classificador de questões na
língua inglesa, montado com aprendizado de máquina supervisionado. O principal objetivo deste trabalho foi concluído, uma vez que os resultados obtidos pelo classificador são extremamente satisfatorios para construção de uma FAQ considerando os Objetivos de Desenvolvimento Sustentável (ODS) da ONU.

Podemos considerar os resultados obtidos pelo classificador como extremamente satisfatórios, tendo uma pontuação de 80% durante a classificação da classe bruta e uma média próxima de 95% entre os classificadores secundários, considerando o algoritmo *Random Forest*.

Ao final deste trabalho, diversos caminhos diferentes surgiram para trabalhos futuros, entre as quais se destacam a criação de um agente virtual para apoio ao trabalho dos estudantes, baseado em classificadores hierárquicos para identificar o módulo e submódulo que possui a resposta para a pergunta de um usuário. Além disso também poderia ser explorada a SimpleTransformers. 

## Referências

[1] P. Guitarrara, "Desenvolvimento sustentável", acessado em: 17/07/2021. Disponível em: https://brasilescola.uol.com.br/geografia/desenvolvimento-sustentavel.htm.

[2] UN, "The Sustainable Development Goals Report 2020", acessado em: 17/07/2021. Disponível em: https://unstats.un.org/sdgs/report/2020/The-Sustainable-Development-Goals-Report-2020.pdf.

[3] "NLP Augmentation", acessado em: 17/07/2021. Disponível em: https://github.com/makcedward/nlpaug.

## Contribuições
<br>
O membro do grupo <b>Germán Darío Buitrago Salazar</b> contribuiu com a criação da base de dados, estratégia de tuning de modelos, escrita do relatório, e geração do resultados.

O membro do grupo <b>Giovanna Relvas Bartilotti</b> contribuiu com a criação da base de dados, escrita do relatório, vídeo.

O membro do grupo <b>Lucas Zanco Ladeira</b> contribuiu com o desenvolvimento da solução na parte de augmentação, treinamento e uso do embedding (Doc2Vec), preparação dos dados, modelagem inicial, criação do notebook talk_to_me, e escrita deste relatório.

O membro do grupo <b>Rafael Scherer</b> contribuiu com a criação da base de dados, desenvolvimento e geração dos resultados.

<b>Observações</b>

A ideia inicial do trabalho era de usar um modelo de transformer (BERT, Roberta, etc), mas tivemos alguns problemas com a implementação de bibliotecas. Pensamos em utilizar SimpleTransformers, Pytorch ou Tensorflow, mas pelo tempo disponível achamos melhor utilizar uma arquitetura mais simples da solução.